In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [3]:
covidx_dir = '/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/'
covidx_img_dir= covidx_dir 

print(covidx_dir)
print(covidx_img_dir)

/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/
/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/


In [4]:
def seedfill(img, val):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), val);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), val);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), val);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), val);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    _, contours, hier = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(4, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1

    
    bin_image2 = 1 - bin_image 
    seedfill(bin_image2,1)
  

    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(bin_image2, cmap=plt.cm.bone)     
    
    
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv,255)/255 
    cleared = clear_border(bin_image_inv)
    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(2)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [6]:
#this code is to generate bounding box and percent of lung mask for traning and validation dataset

subsets = ['val']

from tqdm import tqdm

debug = False 

#filerer window 
(xmin,ymin,xmax,ymax) = (120, 240, 370, 340)

for subset in subsets:     
    
    val_annot_file = covidx_dir +"{}_ICCV_MAI2.txt".format(subset) 
    #fp = open(val_annot_file,'w')
    
    cls = ['covid','non-covid'] 
    count = 0 
    
    for c in cls: 
        cls_path = os.listdir(covidx_img_dir + "{}/{}/".format(subset,c))
        num_scans = len(cls_path) 
        
        pbar = tqdm(total=num_scans)
        
        for ind in range(num_scans):            
            
            pbar.update()
            
            s = "ct_scan_{}".format(ind)
            s_path = "{}/{}/{}".format(subset,c,s)
            #print(s_path)
            
            sfiles = os.listdir(covidx_img_dir + s_path) 
            sfiles = [x for x in sfiles if not x.startswith('.')]
            sfiles.sort(key = lambda x: int(x.split('.')[0]))
                    
            count = 0 
            for f in sfiles:
                count +=1 
                fn = "{}/{}/{}/{}".format(subset,c,s,f)
                print(fn)
                
                f_full = covidx_img_dir + fn
                img = cv2.imread(f_full, cv2.IMREAD_UNCHANGED)
                mask, (xmin, ymin, xmax, ymax) = auto_body_crop(img,plot=False)
                
                #auto_segment(img,plot=True)
                
                
                img_crop = img[ymin:ymax,xmin:xmax] 
                mask_crop = mask[ymin:ymax,xmin:xmax] 
                
                img_crop_2 = img_crop.copy()
                #img_crop_2 = 255 - img_crop_2                
                mask_crop_ind = (mask_crop == 0) 
                img_crop_2[mask_crop_ind] = 0 
                img_crop_3 = seedfill(img_crop_2,255)
                
                rows,cols = img_crop_3.shape
                percent_dark = np.sum(img_crop_3 <10)/rows/cols 
                #print(percent_dark)
                if True: 
                    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(5,1,figsize=(5, 40))
                    ax1.imshow(img,cmap='gray')                    
                    ax2.imshow(img_crop,cmap='gray')
                    ax3.imshow(img_crop_2,cmap='gray')
                    ax4.imshow(img_crop_3,cmap='gray')               
                    ax5.imshow(mask_crop,cmap='gray')               
                    
                    break 
                

                count += 1             
                #fp.write("{} {} {} {} {} {} {:2f}\n".format(fn,c,xmin,ymin,xmax,ymax,percent_dark ))  
            break
        break
                
                
    #fp.close()


  0%|          | 0/165 [00:00<?, ?it/s]

In [14]:
#this code is to generate bounding box and percent of lung mask for test dataset
from tqdm import tqdm

subsets = ['test']

debug = False 

c = 'covid' #all unknown for test dataset 

for subset in subsets: 
    
    #use the following code on test datset 
    testsubsets = ["subset{}".format(x) for x in range(1,9)]
    print(testsubsets) 

    annot_file = covidx_img_dir+ "{}_ICCV_MAI2.txt".format(subset) 
    fp = open(annot_file,'w')
    
    annots = dict()
    
    for test in testsubsets:     

        print(test)
        
        annots[test] = [] 
        
        subset_dir = 'test/' + test     
        #print(subset_dir)    
        scan_dirs = os.listdir(covidx_img_dir + subset_dir)
        
        pbar = tqdm(total=len(scan_dirs))
        
        for s in scan_dirs:
            
            pbar.update() 
            s_dir = subset_dir  + '/' + s         
            #print(s_dir)         
            sfiles = os.listdir(covidx_img_dir + s_dir) 
            sfiles = [x for x in sfiles if not x.startswith('.')]        
            sfiles.sort(key=lambda x: int(x.split('.')[0]) )        
            #print(files)        
            
            count = 0 
            for f in sfiles:
                count +=1 
                fn = s_dir + '/' + f 
                f_full = covidx_img_dir + fn 
                
                img = cv2.imread(f_full, cv2.IMREAD_UNCHANGED)
                mask, (xmin, ymin, xmax, ymax) = auto_body_crop(img,plot=False)
                #auto_segment(img,plot=True)
                
                img_crop = img[ymin:ymax,xmin:xmax] 
                mask_crop = mask[ymin:ymax,xmin:xmax] 
                
                img_crop_2 = img_crop.copy()
                #img_crop_2 = 255 - img_crop_2                
                mask_crop_ind = (mask_crop == 0) 
                img_crop_2[mask_crop_ind] = 0 
                img_crop_3 = seedfill(img_crop_2,255)
                
                rows,cols = img_crop_3.shape
                percent_dark = np.sum(img_crop_3 <10)/rows/cols 
                #print(percent_dark)
                if debug: 
                    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(5,1,figsize=(5, 40))
                    ax1.imshow(img,cmap='gray')                    
                    ax2.imshow(img_crop,cmap='gray')
                    ax3.imshow(img_crop_2,cmap='gray')
                    ax4.imshow(img_crop_3,cmap='gray')               
                    ax5.imshow(mask_crop,cmap='gray') 
                    
                count += 1             
                fp.write("{} {} {} {} {} {} {:2f}\n".format(fn,c,xmin,ymin,xmax,ymax,percent_dark))  
                
                annots[test].append("{} {} {} {} {} {} {:2f}\n".format(fn,c,xmin,ymin,xmax,ymax,percent_dark))
                
                
    fp.close()









  0%|          | 0/450 [00:00<?, ?it/s]

['subset1', 'subset2', 'subset3', 'subset4', 'subset5', 'subset6', 'subset7', 'subset8']
subset1









  0%|          | 2/450 [00:02<08:47,  1.18s/it]






  1%|          | 3/450 [00:02<07:29,  1.01s/it]






  1%|          | 4/450 [00:03<06:38,  1.12it/s]






  1%|          | 5/450 [00:05<08:17,  1.12s/it]






  1%|▏         | 6/450 [00:07<10:05,  1.36s/it]






  2%|▏         | 7/450 [00:10<14:02,  1.90s/it]






  2%|▏         | 8/450 [00:12<14:50,  2.01s/it]






  2%|▏         | 9/450 [00:15<15:51,  2.16s/it]






  2%|▏         | 10/450 [00:17<16:03,  2.19s/it]

KeyboardInterrupt: 